# GAME OF THRONES: A GRAPH EXPERIMENT



### get the extracted characters informations from pickles

The dictionaries are loaded directly

In [48]:
import pickle
with open('characters.pickle','rb') as fp:
    characters = pickle.load(fp)
#print(characters)    

In [49]:
with open('characters_links.pickle','rb') as fp:
    character_links = pickle.load(fp)
    ccc 

## Read the chapters and the book they belong to
The books have been cut into chapters themselves into a book directory named GOT{x} where x is the book in order of publication.
We read the whole directory to retain only text files. we create a chapter list containing the text of the chapter and the book index

In [50]:
import os
from os.path import join

chapters = []

for root, dirs, files in os.walk('data'):
    for name in files:
        if name.endswith('txt'):
            book = int(root.split('/')[-1][-1:])
            with open(join(root,name),'r') as fp:
                chapters.append((book,fp.read()))
                
print('found',len(chapters),'chapters')
#print(chapters[1])

found 345 chapters


## For each Chapter, we analyze the entities and prepare the sentences
Each chapter contains some Persons, that might be characters. We save them and their position together with the book they belong to. 
This will allow us to 
* get a distance between persons
* disambiguate the characters using the book they appear in to

In [51]:
#get candidate fonction. Call for lemmatize
def getCandidates(name, shortened, key, equal=True):
    candidates = []
    print(name, key, equal)
    for s in shortened:
        if key != 'aliases':
            if equal and s[key] is not None:
                if name == s[key]:
                    candidates.append(s)
            elif not equal and s[key] is not None:
                if name in s[key]:
                    candidates.append(s)
        else:
            if equal and s['aliases'] is not None:
                for a in s['aliases']:
                    if name == a:
                        candidates.append(s)
            elif not equal and s['aliases'] is not None:
                for a in s['aliases']:
                    if name in a:
                        candidates.append(s)
    
    #popularity
    if len(candidates)>0:
        if len(candidates)>1:
            mvp = sorted(candidates, key=lambda c: len(c['aliases']), reverse=True)[0]
            return mvp['common_name']
        else:
            return candidates[0]['common_name']
    else:
        None

#lemmatize fonction        
def lemmatize(name, book):
    shortened = []
    for c in characters:
        if c['books'] is not None:
            if book in c['books']:
                shortened.append(c)
    
    priorities = ['common_name', 'short_name', 'fullname', 'aliases']
    
    #try in priorities
    for priority in priorities:
        candidate = getCandidates(name, shortened, priority)
        if candidate is not None:
            return candidate
    
    #try inside priorities
    for priority in priorities:
        candidate = getCandidates(name, shortened, priority, equal=False)
        if candidate is not None:
            return candidate
    
    return None

In [52]:
print(lemmatize('Ned',2))

Ned common_name True
Ned short_name True
Ned fullname True
Ned aliases True
Eddard Stark


In [56]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import networkx as nx

STOPLIST = set(["n't", "'s", "'m", "ca"] + list(STOP_WORDS))
#SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", '\t','\n']
nlp = spacy.load('en_core_web_sm')

final_characters = {}
sentences = [] 
for i in range(len(chapters)):
    if i > 2:
        break
    book, chapter = chapters[i]
    chapter = chapter.strip().replace("\n", " ").replace("\t", " ")
    doc = nlp(chapter)

    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            entity = ent.text.replace("'s",'')
            #lemmatize
            if ent.text not in final_characters.keys():
                final_characters[entity] = {'books':set(),'positions':[]}
            
            final_characters[entity]['books'].add(book)
            final_characters[entity]['positions'].append((ent.start, ent.end))
            ent.merge(ent.root.tag_,ent.text, ent.label_)
    
    sentence = []
    for token in doc:
       #print(token.ent_type_, token.text, token.pos_)
        if token.ent_type_ == 'PERSON':
            text = token.text.replace(' ', '_').replace("'s",'')
            tag = token.ent_type_
            sentence.append('%s|%s' % (text, tag))
        elif token.pos_ not in ['PUNCT','SPACE']:
            sentence.append(token.lemma_.strip() if token.lemma_ != "-PRON-" else token.lower_)
        elif token.pos_ == 'PUNCT' and token.text == '.':
            sentences.append(sentence)
            sentences.append([word for word in sentence if word not in STOPLIST])
            sentence = []

print(final_characters.keys())

dict_keys(['Waymar Royce', 'Gared', 'Royce', 'Ser Waymar Royce', 'Ser Waymar', 'Sworn', 'Bet', 'Mormont', 'Mallister', 'Mallisters', 'Will', 'Frost', 'Maester Aemon', 'Twilight', 'Down', 'Pale', 'Waymar', 'Tall', 'Robert', 'Would Gared', 'Bran', 'Mance Rayder', 'Robb', 'Jon', 'Eddard Stark', 'Jory Cassel', 'Eddard', 'Jon Snow', 'Ass', 'Race', 'Targaryen', 'Starks', 'Jory', 'Harwin', 'Stark', 'Snow', 'Desmond', 'Lady Stark', 'Something', 'Catelyn Tully Stark', 'Uncle Benjen', 'Ned', 'Grey Wind', 'Arya', 'Ghost', 'Septa Mordane', 'Mikken', 'Pentos', 'Myr', 'Run', 'Uncle Ben'])


In [32]:
from gensim.models import Word2Vec
model = Word2Vec(sentences, size=50, window=5, min_count=1, workers=4)
model.wv.most_similar('Bran|PERSON')

C:\Users\lolal\Anaconda3\lib\site-packages\gensim-3.6.0-py3.7-win-amd64.egg\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('he', 0.9986662864685059),
 ('his', 0.9986526370048523),
 ('the', 0.9985457062721252),
 ('and', 0.9985151290893555),
 ('of', 0.9984903335571289),
 ('man', 0.99846351146698),
 ('a', 0.9984533786773682),
 ('as', 0.9984182119369507),
 ('to', 0.9983260631561279),
 ('be', 0.9982801675796509)]

In [33]:
import itertools
couples = list(itertools.combinations(characters, 2))
G.add_edges_from(couples)

NameError: name 'G' is not defined

In [ ]:
print(nx.info(G))
nx.write_gexf(G, 'one-chapter.gexf')